In [31]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

In [36]:
data = pd.read_csv("train.csv",names=["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","output"])
li = ["workclass","education","marital-status","occupation","relationship","race","sex","native-country","output"]
for i in li:
    data[i] = data[i].astype('category')
    data[i] = data[i].cat.codes
# data = data.to_numpy()
y = data.iloc[:,14]
x = data.iloc[:,0:14]
y = y.to_numpy()
x = x.to_numpy()
